В файле <i>orders</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта, 
    price - цена единицы товара, 
    quantity - количество единиц товара в заказе
```
В файле <i>categories</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта

## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

* Определите, какого числа было сделано больше всего заказов?
* Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)
* Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

In [1]:
# Импорт библиотек
import pandas as pd
import json

In [2]:
categories = pd.read_csv('data/categories.csv')
orders = pd.read_json('data/orders.json')

In [3]:
# Вопрос 1: Определение даты с наибольшим количеством заказов

orders['OrderDate'] = pd.to_datetime(orders['OrderTime']).dt.date
order_counts = orders['OrderDate'].value_counts()

max_orders_date = order_counts.idxmax()
max_orders_count = order_counts.max()

print(f"Дата с наибольшим количеством заказов: {max_orders_date} ({max_orders_count} заказов)")

Дата с наибольшим количеством заказов: 2021-02-05 (342 заказов)


In [4]:
# Вопрос 2:ТОП-3 ParentCategory по GMV

# Записываю список 'items' в отдельные строки
orders_expanded = orders.explode('items')


In [5]:
# Извлекаю инфу о товарах
orders_expanded['ItemID'] = orders_expanded['items'].apply(lambda x: x['ItemID'])
orders_expanded['price'] = orders_expanded['items'].apply(lambda x: x['price'])
orders_expanded['quantity'] = orders_expanded['items'].apply(lambda x: x['quantity'])

In [6]:
# Привожу данные к одному формату
orders_expanded['ItemID'] = orders_expanded['ItemID'].astype(str)
categories['ItemID'] = categories['ItemID'].astype(str)

In [7]:
# Объединяю с категориями по 'ItemID'
merged_data = orders_expanded.merge(categories, on='ItemID', how='left')

In [8]:
# Считаю GMV
merged_data['GMV'] = merged_data['price'] * merged_data['quantity']

# Группирую по ParentCategory и считаю GMV
gvm_summary = merged_data.groupby('ParentCategory').agg({'GMV': 'sum'}).reset_index()

In [9]:
# Результат
top_3_categories = gvm_summary.nlargest(3, 'GMV')

print("Топ-3 ParentCategory по GMV:")
print(top_3_categories)

Топ-3 ParentCategory по GMV:
        ParentCategory          GMV
11   Мясо, птица, рыба  32268493.46
9   Молочная продукция  27702995.43
13      Овощи и фрукты  27607486.90


In [11]:
# Вопрос 3: Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

def recommend_item(order_id):
    order_data = orders[orders['OrderID'] == order_id]
    
    if order_data.empty:
        return "Заказ не найден."
    
    items_in_order = order_data['items'].values[0]
    
    # Преобразую строку JSON в список
    items_in_order = json.loads(items_in_order)
    
    # Проверяю, является ли items списком
    if isinstance(items_in_order, list):
        item_ids = [item['ItemID'] for item in items_in_order]
    else:
        return "Формат данных о товарах некорректен."
    
    return item_ids

# Вызываю функцию
order_id_to_check = 12345
recommended_items = recommend_item(order_id_to_check)

# Проверяю работу функции
print("Рекомендованные ItemID для заказа", order_id_to_check, ":", recommended_items)

Рекомендованные ItemID для заказа 12345 : Заказ не найден.
